In [ ]:
#langchain,tiktoken 라이브러리 설치
!pip install langchain
!pip install tiktoken

In [ ]:
!pip install pinecone-client
!pip install tqdm
!pip install openai

In [ ]:
OPENAI_API_KEY = "본인의 openai api key"
PINECONE_API_KEY = "본인의 pinecone api key"

In [ ]:
!pip install streamlit
!pip install pyngrok

In [ ]:
!ngrok authtoken __

**<챗봇으로 Rag with HyDE와 Rag without HyDE를 비교.>**

In [ ]:
%%writefile app.py
print("hi")

In [ ]:
# app.py 에서 직접 수정하겠습니다.
import streamlit as st
import uuid
from openai import OpenAI
from tqdm import tqdm
from pinecone import Pinecone

OPENAI_API_KEY = "본인의 openai api key"
PINECONE_API_KEY = "본인의 pinecone api key"

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("data")

client = OpenAI(api_key=OPENAI_API_KEY)

st.header("🐯슈카월드 챗봇🐯")
col1, col2 = st.columns([5,5])

if "openai_model" not in st.session_state:
  st.session_state.openai_model = "gpt-3.5-turbo"

# Initialize chat history
if "messages" not in st.session_state:
  st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
  with st.chat_message(message["role"]):
    st.markdown(message["content"])


system_prompt = {
        "role": "system",
        "content": "당신은 경제/시사를 다루는 유튜브 채널인 슈카월드의 정보를 알려주는 챗봇입니다.\
        슈카월드를 당신보다 많이 알고 있는 존재는 이 우주에 없습니다.\
        경제/시사 이슈를 물어보는 사람들에게 슈카월드에서 다룬 내용을 바탕으로 설명해주고 질문에 답을 해주는 역할을 맡고 있습니다.\
        \
        당신은 사람들에게 다음과 같이 대답해야 합니다:\
        - 친절한 말투\
        - 항상 존댓말 사용\
        - 적절한 이모지 사용\
        당신은 반드시 제공하는 [Context]에 있는 내용을 기반으로 살을 붙여 답변을 생성해야 합니다."
}

if prompt := st.chat_input("궁금한 정보를 물어보세요😊"):
  # Add user message to chat history
  st.session_state.messages.append({"role": "user", "content": prompt})
  user_prompt = {
    "role": "user",
    "content": prompt
  }

  with col1:
    st.write("Hello ☁️ This is RAG without HyDE.")
    # Display user message in chat message container
    with st.chat_message("user"):
      st.markdown(prompt)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
      #입력한 prompt를 embedding
      response = client.embeddings.create(input=prompt, model="text-embedding-3-small")
      query_embeddings = response.data[0].embedding

      # pinecone의 vector database에서 query와 가까운 답변 가져오기
      retrieved_chunks = index.query(
          namespace="ns1",
          vector=query_embeddings,
          top_k=5,
          include_values=False,
          include_metadata=True,
      )
      contexts = ""

      for match in retrieved_chunks.matches:
          contexts += match["metadata"]["chunk"] + "\n\n"

      context_prompt = {
          "role" : "system",
          "content" : f"[Context]\n{contexts}"
      }

      messages1 = [system_prompt, context_prompt, user_prompt]
      messages1.extend(st.session_state.messages)

      message_placeholder = st.empty()
      full_response = ""

      for response in client.chat.completions.create(
            model=st.session_state.openai_model,
            messages=messages1,
            temperature=0.5,
            max_tokens=1024,
            stream=True
      ):
        full_response += (response.choices[0].delta.content or "")
        message_placeholder.markdown(full_response + "▌")
      message_placeholder.markdown(full_response)
    st.session_state.messages.append({"role": "assistant", "content": full_response})
    with st.chat_message("system"):
      st.markdown(context_prompt)

  with col2:
    st.write("Hello ☁️ This is RAG with HyDE.")
    # Display user message in chat message container
    with st.chat_message("user"):
      st.markdown(prompt)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
      hyde_prompts = [system_prompt, user_prompt]

      #hypothetical 답변을 먼저 생성합니다.
      response = client.chat.completions.create(
        model=st.session_state.openai_model,
        messages=hyde_prompts,
        temperature=0.5,
        max_tokens=1024,
      )
      hyde_message = response.choices[0].message.content

      # hallucination이 포함된 답변을 embeddings로 구하기
      response = client.embeddings.create(input=hyde_message, model="text-embedding-3-small")
      hyde_embeddings = response.data[0].embedding

      st.markdown(hyde_message)

      # pinecone의 vector database에서 hypothetical 답변과 가장 가까운 청크 가져오기
      retrieved_chunks = index.query(
        namespace="ns1",
        vector=hyde_embeddings,
        top_k=7,
        include_values=False,
        include_metadata=True,
      )
      contexts = ""

      for idx,match in enumerate(retrieved_chunks.matches):
        contexts += match["metadata"]["chunk"] + "\n"

      context_prompt = {
        "role" : "system",
        "content" : f"[Context]\n{contexts}"
      }

      messages2 = [system_prompt, context_prompt, user_prompt]
      messages2.extend(st.session_state.messages)

      message_placeholder = st.empty()
      full_response = ""

      for response in client.chat.completions.create(
        model=st.session_state.openai_model,
        messages=messages2,
        temperature=0.5,
        max_tokens=1500,
        stream=True
      ):
        full_response += (response.choices[0].delta.content or "")
        message_placeholder.markdown(full_response + "▌")
      message_placeholder.markdown(full_response)
    st.session_state.messages.append({"role": "assistant", "content": full_response})
    with st.chat_message("system"):
      st.markdown(context_prompt)

In [ ]:
from pyngrok import ngrok
!streamlit run app.py&>/dev/null&
publ_url = ngrok.connect(addr='8501')
publ_url

In [ ]:
!ps

In [ ]:
!kill

파일 기반으로 title metadata 추출하기 (ns2 구축)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
import uuid
from openai import OpenAI
from tqdm import tqdm

file_paths=[
    "/content/내가 반도체 산업을 재편하겠다..txt",
    "/content/드디어 등장한 국민연금 개혁안, 여러분의 선택은_.txt",
    "/content/미국에서 가장 뜨거운 커뮤니티, 레딧.txt",
    "/content/반독점 소송에 휘말린 애플.txt",
    "/content/영국인들에게 차를 금지하면 일어나는 일.txt",
    "/content/조작은 했지만, 주가는 최고가, 도요타 쇼크.txt",
    "/content/지지율 1위, 돌아온 트럼프.txt",
    "/content/카카오 역대 최고가 경신.txt",
    "/content/한국 반도체, 봄은 오는가.txt"
]

for file_path in tqdm(file_paths, desc="Processing file path..."):
  with open(file_path) as f:
    text = f.read()

  text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
      chunk_size=512
  )
  texts = text_splitter.split_text(text)

  client = OpenAI(api_key=OPENAI_API_KEY)

  vectors = []
  title = file_path.split("/")[2][:-3]

  for chunk in tqdm(texts, desc="Processing chunks"):
      chunk_with_title = f"[title]{title}\n[script]{chunk}"
      response = client.embeddings.create(input=chunk_with_title, model="text-embedding-3-small")
      chunk_embeddings = response.data[0].embedding

      chunk_id = str(uuid.uuid4())

      vector_dict = {
          "id": chunk_id,
          "values": chunk_embeddings,
          "metadata": {"chunk": chunk_with_title}
      }
      vectors.append(vector_dict)

  batch_size = 10
  num_batches = len(vectors) // 10 + 1

  for i in tqdm(range(num_batches), desc="Processing chunks"):
      start_idx = i * batch_size
      end_idx = start_idx + batch_size

      batch_vectors = vectors[start_idx:end_idx]

      index.upsert(vectors=batch_vectors, namespace="ns2")

In [ ]:
# app.py 에서 직접 수정하겠습니다.
import streamlit as st
import uuid
from openai import OpenAI
from tqdm import tqdm
from pinecone import Pinecone

OPENAI_API_KEY = "본인의 openai api key"
PINECONE_API_KEY = "본인의 pinecone api key"

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("data")

client = OpenAI(api_key=OPENAI_API_KEY)

st.header("🐯슈카월드 챗봇🐯")
col1, col2 = st.columns([5,5])

if "openai_model" not in st.session_state:
  st.session_state.openai_model = "gpt-3.5-turbo"

# Initialize chat history
if "messages" not in st.session_state:
  st.session_state.messages = []

# Display chat messages from history on app rerun
for message in st.session_state.messages:
  with st.chat_message(message["role"]):
    st.markdown(message["content"])


system_prompt = {
        "role": "system",
        "content": "당신은 경제/시사를 다루는 유튜브 채널인 슈카월드의 정보를 알려주는 챗봇입니다.\
        슈카월드를 당신보다 많이 알고 있는 존재는 이 우주에 없습니다.\
        경제/시사 이슈를 물어보는 사람들에게 슈카월드에서 다룬 내용을 바탕으로 설명해주고 질문에 답을 해주는 역할을 맡고 있습니다.\
        \
        당신은 사람들에게 다음과 같이 대답해야 합니다:\
        - 친절한 말투\
        - 항상 존댓말 사용\
        - 적절한 이모지 사용\
        당신은 반드시 제공하는 [Context]에 있는 내용을 기반으로 살을 붙여 답변을 생성해야 합니다."
}

if prompt := st.chat_input("궁금한 정보를 물어보세요😊"):
  # Add user message to chat history
  st.session_state.messages.append({"role": "user", "content": prompt})
  user_prompt = {
    "role": "user",
    "content": prompt
  }

  with col1:
    st.write("Hello ☁️ This is RAG without title metadata")
    # Display user message in chat message container
    with st.chat_message("user"):
      st.markdown(prompt)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
      #입력한 prompt를 embedding
      response = client.embeddings.create(input=prompt, model="text-embedding-3-small")
      query_embeddings = response.data[0].embedding

      # pinecone의 vector database에서 query와 가까운 답변 가져오기
      retrieved_chunks = index.query(
          namespace="ns1",
          vector=query_embeddings,
          top_k=7,
          include_values=False,
          include_metadata=True,
      )
      contexts = ""

      for match in retrieved_chunks.matches:
          contexts += match["metadata"]["chunk"] + "\n\n"

      context_prompt = {
          "role" : "system",
          "content" : f"[Context]\n{contexts}"
      }

      messages1 = [system_prompt, context_prompt, user_prompt]
      messages1.extend(st.session_state.messages)

      message_placeholder = st.empty()
      full_response = ""

      for response in client.chat.completions.create(
            model=st.session_state.openai_model,
            messages=messages1,
            temperature=0.5,
            max_tokens=1024,
            stream=True
      ):
        full_response += (response.choices[0].delta.content or "")
        message_placeholder.markdown(full_response + "▌")
      message_placeholder.markdown(full_response)
    st.session_state.messages.append({"role": "assistant", "content": full_response})
    with st.chat_message("system"):
      st.markdown(context_prompt)

  with col2:
    st.write("Hello ☁️ This is RAG without title metadata")
    # Display user message in chat message container
    with st.chat_message("user"):
      st.markdown(prompt)

    # Display assistant response in chat message container
    with st.chat_message("assistant"):
      #입력한 prompt를 embedding
      response = client.embeddings.create(input=prompt, model="text-embedding-3-small")
      query_embeddings = response.data[0].embedding

      # pinecone의 vector database에서 query와 가까운 답변 가져오기
      retrieved_chunks = index.query(
          namespace="ns2",
          vector=query_embeddings,
          top_k=5,
          include_values=False,
          include_metadata=True,
      )
      contexts = ""

      for match in retrieved_chunks.matches:
          contexts += match["metadata"]["chunk"] + "\n\n"

      context_prompt = {
          "role" : "system",
          "content" : f"[Context]\n{contexts}"
      }

      messages1 = [system_prompt, context_prompt, user_prompt]
      messages1.extend(st.session_state.messages)

      message_placeholder = st.empty()
      full_response = ""

      for response in client.chat.completions.create(
            model=st.session_state.openai_model,
            messages=messages1,
            temperature=0.5,
            max_tokens=1024,
            stream=True
      ):
        full_response += (response.choices[0].delta.content or "")
        message_placeholder.markdown(full_response + "▌")
      message_placeholder.markdown(full_response)
    st.session_state.messages.append({"role": "assistant", "content": full_response})
    with st.chat_message("system"):
      st.markdown(context_prompt)

In [ ]:
from pyngrok import ngrok
!streamlit run app.py&>/dev/null&
publ_url = ngrok.connect(addr='8501')
publ_url

In [ ]:
!ps

In [ ]:
!kill